In [1]:
%load_ext autoreload
%autoreload 2 
#default_exp utils

In [2]:
#exporti
from fastai.tabular.all import * 
from tabnet.model import * 

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Model creating functions

### classifier

In [15]:
#export
@delegates(TabNetBase.__init__)
def TabNetClassifier(head_func, to, **kwargs):
    return TabNet(head_func, emb_szs=get_emb_sz(to), n_cont=len(to.cont_names), n_out=to.c, **kwargs)

### self supervised

In [16]:
#export
@delegates(TabNetBase.__init__)
def TabNetSelfSupervised(head_func, bs=1024, **kwargs):
    n_out = len(get_emb_sz(to)) + len(to.cont_names)
    return TabNet(tabnet_decoder, emb_szs=get_emb_sz(to), n_cont=len(to.cont_names), n_out=n_out, **kwargs)

# Self Supervised Data Loader 

In [17]:
#export
class ReadTabBatchIdentity(ItemTransform):
    
    def __init__(self, to): store_attr()
        
    def encodes(self, to):
        if not to.with_cont: res = (tensor(to.cats).long(),)
        else: res = (tensor(to.cats).long(),tensor(to.conts).float())
        res = res + res #
        if to.device is not None: res = to_device(res, to.device)
        return res 

In [18]:
#export
class TabularPandasIdentity(TabularPandas): pass 

In [19]:
#export
@delegates()
class TabDataLoaderIdentity(TabDataLoader):
    do_item = noops
    def __init__(self, dataset, bs=16, shuffle=False, after_batch=None, num_workers=0, **kwargs):
        if after_batch is None: after_batch = L(TransformBlock().batch_tfms)+ReadTabBatchIdentity(dataset)
        super().__init__(dataset, bs=bs, shuffle=shuffle, after_batch=after_batch, num_workers=num_workers, **kwargs)

    def create_batch(self, b): return self.dataset.iloc[b]

TabularPandasIdentity._dl_type = TabDataLoaderIdentity

# Export

In [14]:
from nbdev.export import notebook2script
notebook2script()

Converted 01_core.ipynb.
Converted 02_model.ipynb.
No export destination, ignored:
#exporti
from fastai.tabular.all import * 
from tabnet.utils import *
from tabnet.model import *
Converted 03_experiments.ipynb.
Converted 04_self_supervision.ipynb.
Converted 04_utils.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.
